In [39]:
import numpy as np
import pandas as pd
import re

import nltk


In [7]:
df = pd.read_csv("data/iowaliquor.csv")

/Users/anishyakkala/Library/Python/3.6/lib/python/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (6,14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
df.head()

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,...,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons)
0,S08096000008,10/04/2012,4641,Kum & Go #573 / SE 14th DM,5830 SE 14th ST,DES MOINES,50315,NaN,77.0,Polk,...,89191,Jose Cuervo Especial Reposado Tequila Mini,12,500,11.50,17.25,1,17.25,0.50,0.13
1,S23102300041,12/20/2014,4346,Roy's Foodland,105 PEARL ST,SHELLSBURG,52332,POINT (-91.869285 42.094155),6.0,Benton,...,11774,Black Velvet,24,375,3.07,4.61,6,27.66,2.25,0.59
2,S14410500035,09/09/2013,3628,Wal-Mart 1528 / Cedar Rapids,2645 BLAIRS FERRY RD NE,CEDAR RAPIDS,52402,POINT (-91.680734 42.034748),57.0,Linn,...,43410,Captain Morgan Parrot Bay Coconut,12,750,7.49,11.23,12,134.76,9.00,2.38
3,S09427600021,12/12/2012,4708,No Frills Supermarkets #803 / Glenwo,423 SHARP ST,GLENWOOD,51534,POINT (-95.742987 41.04635),65.0,Mills,...,24458,Kessler Blend Whiskey,6,1750,11.01,16.52,30,495.60,52.50,13.87
4,S28446900114,10/14/2015,2594,Hy-Vee Food Store / Sioux City,4500 SERGEANT ROAD,SIOUX CITY,51106,POINT (-96.346969 42.447396),97.0,Woodbury,...,10550,Black Velvet Toasted Caramel,12,750,6.75,10.13,12,121.56,9.00,2.38


In [15]:
df_raw_county = pd.read_csv("data/co-est2019-alldata.csv", encoding = "ISO-8859-1")
df_raw_county["id"] = df_raw_county.index

In [16]:
df_county_year = pd.wide_to_long(df_raw_county.drop(["SUMLEV","REGION","DIVISION","STATE","COUNTY","CENSUS2010POP","ESTIMATESBASE2010"],axis = 1),
                  ["POPESTIMATE","NPOCHG_","BIRTHS",
                  "DEATHS","NATURALINC","INTERNATIONALMIG",
                  "DOMESTICMIG","NETMIG","RESIDUAL",
                  "GQESTIMATESBASE","RBIRTH","RDEATH","RNATURALINC",
                  "RINTERNATIONALMIG","RDOMESTICMIG","RNETMIG","NPOPCHG_","GQESTIMATES"], i="id", j="year")

In [17]:
df_county_year["CTYNAME"] = df_county_year["CTYNAME"].str.replace(" County", "")

In [18]:
df_county_year.tail()

STNAME CTYNAME  POPESTIMATE  NPOCHG_  BIRTHS  DEATHS  NATURALINC  \
id   year                                                                      
3192 2015  Wyoming  Weston         7208      NaN      74      68           6   
     2016  Wyoming  Weston         7220      NaN      77      69           8   
     2017  Wyoming  Weston         6968      NaN      73      69           4   
     2018  Wyoming  Weston         6924      NaN      67      64           3   
     2019  Wyoming  Weston         6927      NaN      61      75         -14   

           INTERNATIONALMIG  DOMESTICMIG  NETMIG  RESIDUAL  GQESTIMATESBASE  \
id   year                                                                     
3192 2015                42           24      66        -2              NaN   
     2016                14           -9       5        -1              NaN   
     2017                18         -275    -257         1              NaN   
     2018                17          -65     -48         1              NaN   
     2019                18            0      18        -1              NaN   

              RBIRTH     RDEATH  RNATURALINC  RINTERNATIONALMIG  RDOMESTICMIG  \
id   year                                                                       
3192 2015  10.316465   9.479994     0.836470           5.855291      3.345880   
     2016  10.673690   9.564735     1.108955           1.940671     -1.247574   
     2017  10.290386   9.726529     0.563857           2.537356    -38.765154   
     2018   9.645839   9.213936     0.431903           2.447452     -9.357904   
     2019   8.808028  10.829543    -2.021515           2.599090      0.000000   

             RNETMIG  NPOPCHG_  GQESTIMATES  
id   year                                    
3192 2015   9.201171        70          328  
     2016   0.693097        12          319  
     2017 -36.227798      -252          332  
     2018  -6.910452       -44          315  
     2019   2.599090         3          315

In [19]:
df["Year"] = pd.DatetimeIndex(df['Date']).year

In [20]:
df_county_year = df_county_year.reset_index()

In [24]:
df_full = df.merge(df_county_year, 'left', left_on = ['Year','County'], right_on = ['year','CTYNAME'])

In [26]:
df_full.columns


Index(['Invoice/Item Number', 'Date', 'Store Number', 'Store Name', 'Address',
       'City', 'Zip Code', 'Store Location', 'County Number', 'County',
       'Category', 'Category Name', 'Vendor Number', 'Vendor Name',
       'Item Number', 'Item Description', 'Pack', 'Bottle Volume (ml)',
       'State Bottle Cost', 'State Bottle Retail', 'Bottles Sold',
       'Sale (Dollars)', 'Volume Sold (Liters)', 'Volume Sold (Gallons)',
       'Year', 'id', 'year', 'STNAME', 'CTYNAME', 'POPESTIMATE', 'NPOCHG_',
       'BIRTHS', 'DEATHS', 'NATURALINC', 'INTERNATIONALMIG', 'DOMESTICMIG',
       'NETMIG', 'RESIDUAL', 'GQESTIMATESBASE', 'RBIRTH', 'RDEATH',
       'RNATURALINC', 'RINTERNATIONALMIG', 'RDOMESTICMIG', 'RNETMIG',
       'NPOPCHG_', 'GQESTIMATES'],
      dtype='object')

In [44]:
from collections import Counter
Counter(" ".join(df["Item Description"]).split()).most_common(10)


[('Vodka', 3693785),
 ('Rum', 1648831),
 ('Mini', 1423998),
 ('Black', 1318868),
 ('Whiskey', 995637),
 ('Spiced', 967277),
 ('Crown', 855613),
 ('Velvet', 763560),
 ('Liqueur', 748869),
 ('Canadian', 734152)]

In [46]:
vodka_df = df[df["Item Description"].str.contains("Vodka")]

In [69]:
df_pivot_table = pd.pivot_table(df, index = ['County', 'Year','Item Description','Pack', 'Category Name'],
               values = ['Bottle Volume (ml)',
               'State Bottle Cost', 'State Bottle Retail', 'Bottles Sold',
               'Sale (Dollars)', 'Volume Sold (Liters)', 'Volume Sold (Gallons)'],
              aggfunc = np.mean)

In [70]:
df_pivot_table

Bottle Volume (ml)  \
County Year Item Description                  Pack Category Name                                   
ADAIR  2016 1800 Silver Tequila               12   100% Agave Tequila                 750.000000   
            Absolut Swedish Vodka 80 Prf      12   Imported Vodkas                    750.000000   
            Absolut Swedish Vodka 80 Prf Mini 10   Imported Vodka                     600.000000   
                                                   Imported Vodkas                    600.000000   
            Admiral Nelson Cherry Spiced Rum  12   Spiced Rum                         750.000000   
...                                                                                          ...   
Wright 2016 Uv Red (cherry) Vodka             12   VODKA FLAVORED                     791.666667   
            Uv Vodka                          12   VODKA 80 PROOF                     750.000000   
            Viniq                             12   PEPPERMINT SCHNAPPS                750.000000   
            Windsor Canadian                  12   CANADIAN WHISKIES                 1000.000000   
            Woodford Reserve Bourbon          6    STRAIGHT BOURBON WHISKIES          750.000000   

                                                                              Bottles Sold  \
County Year Item Description                  Pack Category Name                             
ADAIR  2016 1800 Silver Tequila               12   100% Agave Tequila             1.000000   
            Absolut Swedish Vodka 80 Prf      12   Imported Vodkas               10.857143   
            Absolut Swedish Vodka 80 Prf Mini 10   Imported Vodka                 2.000000   
                                                   Imported Vodkas                5.166667   
            Admiral Nelson Cherry Spiced Rum  12   Spiced Rum                     1.500000   
...                                                                                    ...   
Wright 2016 Uv Red (cherry) Vodka             12   VODKA FLAVORED                 3.666667   
            Uv Vodka                          12   VODKA 80 PROOF                 5.000000   
            Viniq                             12   PEPPERMINT SCHNAPPS            4.000000   
            Windsor Canadian                  12   CANADIAN WHISKIES              4.000000   
            Woodford Reserve Bourbon          6    STRAIGHT BOURBON WHISKIES      6.000000   

                                                                              Sale (Dollars)  \
County Year Item Description                  Pack Category Name                               
ADAIR  2016 1800 Silver Tequila               12   100% Agave Tequila              23.190000   
            Absolut Swedish Vodka 80 Prf      12   Imported Vodkas                187.177143   
            Absolut Swedish Vodka 80 Prf Mini 10   Imported Vodka                  11.880000   
                                                   Imported Vodkas                 61.380000   
            Admiral Nelson Cherry Spiced Rum  12   Spiced Rum                      13.005000   
...                                                                                      ...   
Wright 2016 Uv Red (cherry) Vodka             12   VODKA FLAVORED                  36.000000   
            Uv Vodka                          12   VODKA 80 PROOF                  48.750000   
            Viniq                             12   PEPPERMINT SCHNAPPS             60.000000   
            Windsor Canadian                  12   CANADIAN WHISKIES               40.880000   
            Woodford Reserve Bourbon          6    STRAIGHT BOURBON WHISKIES      197.940000   

                                                                              State Bottle Cost  \
County Year Item Description                  Pack Category Name                                  
ADAIR  2016 1800 Silver Tequila               12   100% Agave Tequila                 15.460000   
        